In [1]:
# import numpy as np
import pandas as pd
import requests
import datetime
import time

import env

In [6]:
baseUrl = 'https://www.googleapis.com/youtube/v3/search'
youtubeDataBaseParams = {
  'key': env.token,
  'part': 'snippet',
  'order': 'viewCount',
  'regionCode': 'KR',
  'type': 'video'
}

numOfVideoPerMovie = 5
searchKeywords = ['영화', '명장면', '리뷰', '해석', '의미']

In [3]:
movieInfo = pd.read_csv('movieInfo.csv').loc[:,['영화명', '개봉일']]
movieNames = list(movieInfo.loc[:, '영화명'])

In [7]:
# 중복 제거
# 한 키워드당 2개 -> 한 영화당 영상 최대 10개
# 한 영화당 요청 5개 -> 한 영화당 5*20 = 100초 -> 영화 10개당 17분

def req_youtube():
  columns = ['영화 제목', '검색 키워드', '영상 id', '영상 제목', '생성된 날짜']
  res_data = pd.DataFrame(columns=columns)
  tokenNum = 0

  for movieName in movieNames[10:20]:
    print(movieName, len(res_data))
    for keyword in searchKeywords:
      publishedAfter = list(movieInfo[movieInfo['영화명'] == movieName].loc[:, '개봉일'])[0]
      year, month, day = publishedAfter.split('-')
      publishedBefore = (datetime.datetime(int(year), int(month), int(day)) + datetime.timedelta(days=30)).strftime('%Y-%m-%d')
      q = ' '.join([movieName, keyword])

      youtubeDataBaseParams['q'] = q
      youtubeDataBaseParams['publishedAfter'] = publishedAfter+'T00:00:00Z'
      youtubeDataBaseParams['publishedBefore'] = publishedBefore+'T00:00:00Z'

      time.sleep(20)
      res = requests.get(baseUrl, youtubeDataBaseParams)
      data = res.json()

      while 'error' not in data.keys():
        print('[HTTPError] token 교체')
        tokenNum += 1
        if tokenNum > 2: 
          print('[Error] token 오늘 할당량 끝났다~~', movieNames, keyword)
          return res_data

        youtubeDataBaseParams['key'] = env.token[tokenNum]
        res = requests.get(baseUrl, youtubeDataBaseParams)
        data = res.json()

      numOfNewId = 0
      if 'items' in data.keys():
        for item in data['items']:
          res_data_video_ids = list(res_data.loc[res_data['영화 제목'] == movieName,'영상 id'])

          if item['id']['videoId'] not in res_data_video_ids:
            item_data = [[movieName, q, item['id']['videoId'], item['snippet']['title'], item['snippet']['publishTime']]]
            res_data = res_data.append(pd.DataFrame(item_data, columns=columns),ignore_index=True)
            numOfNewId += 1
            if numOfNewId >= 2: break

  return res_data

res_data2 = req_youtube()

분노의 질주: 홉스&쇼 0
[HTTPError] token 교체
토이 스토리 4 10
돈 20
악인전 27
가장 보통의 연애 35
말모이 45
증인 52
블랙머니 60
터미네이터: 다크 페이트 65
사바하 73


In [5]:
res_data

,영화 제목,검색 키워드,영상 id,영상 제목,생성된 날짜
0,극한직업,극한직업 영화,YcMZZ3sjfxk,"극한직업, 닭으로 닥치고 천만 찍어버릴 영화!",2019-02-02T01:01:58Z
1,극한직업,극한직업 영화,dBr-drzet-o,극한직업 [수원 왕갈비 통닭]을 레시피대로 만들어 봤습니다!! [ENG SUB],2019-02-17T05:15:00Z
2,극한직업,극한직업 리뷰,7DRDMuMdxVY,극한직업 수원왕갈비통닭을 먹었습니다.맛상무,2019-02-10T05:45:02Z
3,극한직업,극한직업 리뷰,jFd1X9j_9Os,명절 개봉영화의 역사를 새로쓰는 영화 : &#39;극한직업&#39;리뷰,2019-01-31T13:02:53Z
4,극한직업,극한직업 해석,PwefBCv1sp8,치킨 장사 대박난 마약반 형사들의 어정쩡한 코미디: 극한직업 리뷰,2019-01-25T15:58:45Z
...,...,...,...,...,...
86,82년생 김지영,82년생 김지영 해석,EUlwbK2ZiDw,&#39;82년생 김지영&#39; 악플러 뼈때리는 영화인 이유,2019-11-01T14:48:47Z
87,82년생 김지영,82년생 김지영 의미,NnF0EXqY8As,&#39;82년생 김지영&#39;이 불편한 당신에게 / SBS / 더저널리스트 / ...,2019-11-08T09:37:39Z
88,82년생 김지영,82년생 김지영 의미,UFpk-lXe92M,[이슈 완전정복] 논란 속 개봉 &#39;82년생 김지영&#39;…흥행·평가는? (...,2019-11-01T06:17:50Z
89,82년생 김지영,82년생 김지영 명장면,GmYXxnR0-Yo,박스오피스 1위 &#39;82년생 김지영&#39;. 해외 반응 총정리 (feat. ...,2019-10-27T02:00:00Z


In [8]:
res_data2

,영화 제목,검색 키워드,영상 id,영상 제목,생성된 날짜
0,분노의 질주: 홉스&쇼,분노의 질주: 홉스&쇼 영화,XZEPXeL8yxM,드웨인존슨 앞에서 덩치로 깝치면 벌어지는 일 (황석희 등장),2019-08-22T09:16:40Z
1,분노의 질주: 홉스&쇼,분노의 질주: 홉스&쇼 영화,0jytjkUZUMw,분노의 질주: 홉스&amp;쇼(2019) OST : Better as One.FLA...,2019-08-16T08:24:36Z
2,분노의 질주: 홉스&쇼,분노의 질주: 홉스&쇼 명장면,cKhdU-cPI2g,[분노의 질주: 홉스&amp;쇼] 파이널 예고편,2019-08-14T01:05:30Z
3,분노의 질주: 홉스&쇼,분노의 질주: 홉스&쇼 명장면,C0OXCx8R_Cg,분노의 질주 명장면 DC의 영웅 최강매력 갤가돗,2019-08-20T12:54:40Z
4,분노의 질주: 홉스&쇼,분노의 질주: 홉스&쇼 리뷰,_nUjA8bQ8gQ,[분노의 질주: 홉스&amp;쇼] 환상의 콤비 예고편,2019-08-14T08:32:44Z
...,...,...,...,...,...
77,사바하,사바하 리뷰,46FS6WdZtoY,뱀은 마귀의 상징이다? 당신이 모르는 사바하 동물들의 상징 해설,2019-02-25T09:20:33Z
78,사바하,사바하 리뷰,adMio0d60_8,사바하 해석 : 불교와 기독교 세계관이 합쳐진 영화,2019-02-23T08:04:29Z
79,사바하,사바하 해석,ZFPUdt38nvM,"사바하 해석 편 [줄거리 알려줌] 불교에서 악마는 어떻게 탄생하나? (娑婆訶, Sv...",2019-02-21T08:30:00Z
80,사바하,사바하 의미,zhXrM-ekO3E,"사바하 해석, 아무도 말해 주지 않는 사바하 진짜 의미",2019-02-28T05:27:45Z


In [12]:
result_data = pd.concat([res_data, res_data2])
result_data

,영화 제목,검색 키워드,영상 id,영상 제목,생성된 날짜
0,극한직업,극한직업 영화,YcMZZ3sjfxk,"극한직업, 닭으로 닥치고 천만 찍어버릴 영화!",2019-02-02T01:01:58Z
1,극한직업,극한직업 영화,dBr-drzet-o,극한직업 [수원 왕갈비 통닭]을 레시피대로 만들어 봤습니다!! [ENG SUB],2019-02-17T05:15:00Z
2,극한직업,극한직업 리뷰,7DRDMuMdxVY,극한직업 수원왕갈비통닭을 먹었습니다.맛상무,2019-02-10T05:45:02Z
3,극한직업,극한직업 리뷰,jFd1X9j_9Os,명절 개봉영화의 역사를 새로쓰는 영화 : &#39;극한직업&#39;리뷰,2019-01-31T13:02:53Z
4,극한직업,극한직업 해석,PwefBCv1sp8,치킨 장사 대박난 마약반 형사들의 어정쩡한 코미디: 극한직업 리뷰,2019-01-25T15:58:45Z
...,...,...,...,...,...
77,사바하,사바하 리뷰,46FS6WdZtoY,뱀은 마귀의 상징이다? 당신이 모르는 사바하 동물들의 상징 해설,2019-02-25T09:20:33Z
78,사바하,사바하 리뷰,adMio0d60_8,사바하 해석 : 불교와 기독교 세계관이 합쳐진 영화,2019-02-23T08:04:29Z
79,사바하,사바하 해석,ZFPUdt38nvM,"사바하 해석 편 [줄거리 알려줌] 불교에서 악마는 어떻게 탄생하나? (娑婆訶, Sv...",2019-02-21T08:30:00Z
80,사바하,사바하 의미,zhXrM-ekO3E,"사바하 해석, 아무도 말해 주지 않는 사바하 진짜 의미",2019-02-28T05:27:45Z


In [14]:
result_data.to_csv('./movieVideoList_0_19.csv')